In [77]:
import dask
import dask.dataframe as dd
import uuid

import pandas as pd
import numpy as np
import os

In [78]:
data_folder = '/run/media/ai/drive25/archives/didi/src/scripts/fake_gps/'
out_folder = '../clean_gps/'

In [79]:
def csvfy():
    os.system(f'mkdir {out_folder}')  # make the output folder if it doesn't exist
    files = os.listdir(data_folder)
    cols = ['DRIVER_ID', 'TRIP_ID', 'TIMESTAMP', 'LONGITUDE', 'LATITUDE']

    for i, f in enumerate(files):
        csvfy_aux(f, i, cols)

In [80]:
def process_group(lng, lat):
    lonlat =  set(zip(lng, lat))
    return lonlat

In [81]:
def csvfy_aux(f, i, cols):
    df = dask.dataframe.read_csv(os.path.join(data_folder, f), names=cols)
    df = df.drop(columns=['DRIVER_ID'], axis=1)
    df = df.set_index('TIMESTAMP').persist()
    
    ndf = df.groupby('TRIP_ID').apply(lambda x: process_group(x.LONGITUDE, x.LATITUDE)).to_frame(name='POLYLINE').reset_index()
    ndf.to_csv(os.path.join(out_folder, f), index=False)

In [82]:
csvfy()

<ipython-input-81-cadb98dfeb1d>:6: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  ndf = df.groupby('TRIP_ID').apply(lambda x: process_group(x.LONGITUDE, x.LATITUDE)).to_frame(name='POLYLINE').reset_index()
